<a href="https://colab.research.google.com/github/Eunjung-Cho/python_study/blob/master/crawler/python_crawler/naver_api/shopping/%EB%84%A4%EC%9D%B4%EB%B2%84_API%EB%A1%9C_%EC%A3%BC%EB%B3%84_%EC%87%BC%ED%95%91_%EA%B2%80%EC%83%89%EC%96%B4_%EB%B6%84%EC%84%9D_211221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 API로 주별 쇼핑 검색어 분석

* [네이버 API 링크 참고](https://developers.naver.com/products/service-api/datalab/datalab.md)

> API(Application Progamming Interface): 어플리케이션 소프트웨어 구축 및 통합을 위한 정의 및 프로토콜 세트
사용이유: 제품 또는 서비스가 서로 커뮤니케이션 가능, 어플리케이션 개발 간소화 (시간/비용 절약) 


> 네이버 데이터랩
쇼핑인사이트: 네이터 통합검색의 쇼핑영역, 네이버쇼핑의 검색결과에서 사용자가 클릭한 데이터를 조회할 수 있는 API

- 분야별 통계 -> 재고관리, 마케팅 등에 활용 
- 사업하고 있는 분야에 대한 검색어 클릭 추이 파악 -> 재고 관리 활용
- 사업하고 있는 분야의 주요 구매층의 클릭 추이 파악(연령별,성별, 기기별 클릭 비중과 추이)
※ 분야 조회는 네이버쇼핑 서비스의 URL에서 cat_id 파라미터의 값(cat_id=숫자에서 숫자)으로 조회할 수 있습니다. 예를 들어 https://search.shopping.naver.com/search/category.nhn?cat_id=50000805에서 cat_id 파라미터의 값인 50000805는 패션의류 > 여성의류 > 니트,스웨터 분야를 나타내는 값입니다.
- 상품과 관련된 검색어의 검색 추이와 검색어별 통계를 제공합니다. 검색어 통계는 마케팅 타깃을 설정하는 데 활용할 수 있습니다. 특정 쇼핑 분야에서 클릭이 발생한 검색어의 연령별, 성별, 기기별 세부 통계를 확인할 수 있습니다.

※ 쇼핑인사이트가 제공하는 값은 요청한 기간 중 클릭 횟수가 가장 높은 값을 100으로 두고 나머지 값을 100에 대한 상대적 값으로 표현한 값입니다.

★ 참고레퍼런스

[네이버 검색어 트렌드 및 시각화하기](https://wooiljeong.github.io/python/naver_datalab_open_api/)

[마켓컬리 네이버 검색량 알아보기](https://yobro.tistory.com/175)

[검색어 트렌드 크롤링 툴](https://brunch.co.kr/@sukhyun9673/13)

[네이버 쇼핑 인사이트 API](https://betterinvesting.tistory.com/269)

[코로나 관련 검색어 트랜드 데이터 수집](https://outyj.tistory.com/9)

[쇼핑카테고리 별 트렌드 쉽게 가지고 오기](https://junglow9.tistory.com/33)

## 0. 코랩 세팅 준비

## 필요한 라이브러리

In [2]:
import os
import sys
import urllib.request
import json
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import re

- 브래키츠_조은정 client_id 로
https://developers.naver.com/apps/#/myapps/MPXfJyDQEVDRagoqt2hv/overview


In [12]:
client_id = "MPXfJyDQEVDRagoqt2hv"
client_secret = "zaOlzPsw1j"

import requests 
import json 
import pprint 

url = "https://openapi.naver.com/v1/datalab/shopping/categories" 
payload = {"startDate": "2017-08-01", 
           "endDate": "2017-09-30", 
           "timeUnit": "month", 
           "category": [ {"name": "패션의류", "param": ["50000000"]}, 
                        {"name": "화장품/미용", "param": ["50000002"]}], 
           "device": "pc", 
           "ages": ["20", "30"], 
           "gender": "f"}

headers = {"X-Naver-Client-Id": client_id, "X-Naver-Client-Secret": client_secret, "Content-Type": "application/json"} 
res = requests.post(url, data=json.dumps(payload), headers=headers) 

print(res.status_code) 
aaa = json.loads(res.text) 
pprint.pprint(aaa)


200
{'endDate': '2017-09-30',
 'results': [{'category': ['50000000'],
              'data': [{'period': '2017-08-01', 'ratio': 84.01252},
                       {'period': '2017-09-01', 'ratio': 100}],
              'title': '패션의류'},
             {'category': ['50000002'],
              'data': [{'period': '2017-08-01', 'ratio': 22.21162},
                       {'period': '2017-09-01', 'ratio': 21.54278}],
              'title': '화장품/미용'}],
 'startDate': '2017-08-01',
 'timeUnit': 'month'}


### 위에 코드 데이터프레임화 시키기

## 1. 데이터 수집_네이버 API로 데이터 크롤링

[네이버 개발자 API 참고](https://developers.naver.com/docs/serviceapi/search/shopping/shopping.md#%EC%87%BC%ED%95%91)

In [13]:
import os
import sys
import urllib.request
import json
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

client_id = "EtWtbcxrug5u7VvHRzMr"
client_secret = "GVuA6qjhow"
url = "https://openapi.naver.com/v1/datalab/search";

keyword_groups = [
                  {"groupName": "코로나", "keywords": ["코로나", "covid-19"]},
                  {"groupName": "원피스", "keywords": ["원피스"] }
]

body = json.dumps({
    "startDate": "2021-07-01",
    "endDate": "2021-08-27",
    "timeUnit": "week",
    "keywordGroups": keyword_groups,
    "device":"pc",
    "ages": ["3","4","5"],
    "gender": "f"
}, ensure_ascii=False)

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
    print("-" * 50)
    res = response_body.decode('utf-8')
    res = json.loads(res)
    print(res)
    print("-" * 50)
    results = res["results"]
    print(results)
    for result in results:
      df = pd.DataFrame(data=result["data"])
      df.columns.name = result["title"]
      display(df)
else:
    print("Error Code:" + rescode)

{"startDate":"2021-06-28","endDate":"2021-08-29","timeUnit":"week","results":[{"title":"코로나","keywords":["코로나","covid-19"],"data":[{"period":"2021-06-28","ratio":24.52658},{"period":"2021-07-05","ratio":100},{"period":"2021-07-12","ratio":72.66324},{"period":"2021-07-19","ratio":68.68416},{"period":"2021-07-26","ratio":43.57595},{"period":"2021-08-02","ratio":34.56169},{"period":"2021-08-09","ratio":46.04983},{"period":"2021-08-16","ratio":32.92325},{"period":"2021-08-23","ratio":26.84747}]},{"title":"원피스","keywords":["원피스"],"data":[{"period":"2021-06-28","ratio":1.5912},{"period":"2021-07-05","ratio":1.48263},{"period":"2021-07-12","ratio":1.58133},{"period":"2021-07-19","ratio":1.54608},{"period":"2021-07-26","ratio":1.34022},{"period":"2021-08-02","ratio":1.26901},{"period":"2021-08-09","ratio":1.13506},{"period":"2021-08-16","ratio":0.96797},{"period":"2021-08-23","ratio":1.08148}]}]}
--------------------------------------------------
{'startDate': '2021-06-28', 'endDate': '2021-08

코로나,period,ratio
0,2021-06-28,24.52658
1,2021-07-05,100.00000
2,2021-07-12,72.66324
3,2021-07-19,68.68416
4,2021-07-26,43.57595
5,2021-08-02,34.56169
6,2021-08-09,46.04983
7,2021-08-16,32.92325
8,2021-08-23,26.84747


원피스,period,ratio
0,2021-06-28,1.59120
1,2021-07-05,1.48263
2,2021-07-12,1.58133
3,2021-07-19,1.54608
4,2021-07-26,1.34022
5,2021-08-02,1.26901
6,2021-08-09,1.13506
7,2021-08-16,0.96797
8,2021-08-23,1.08148


### 위의 코드 csv 파일로 저장하고 태블로에서 그래프 살펴보기

In [7]:
query = urllib.parse.quote(input("검색 질의: "))
idx=0
display=100
start=1
end=1000
sort="sim"



#컬럼 정의는 위의 링크의 4.출력결과 테이블 참고
shop_df = pd.DataFrame(columns=("Title", "Link", "Image", "Low price", "High Price", "Mall Name", "Product ID", "Product Type", "Maker", "Brand"))

for start_index in range(start, end, display):
  url = "https://openapi.naver.com/v1/search/shop?query=" + query + "&display=" + str(display) + "&start" + str(start_index) + "&sort=" + sort

  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  request.add_header("Content-Type","application/json")
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode==200):
    response_body = response.read()
    response_dict = json.loads(response_body.decode('utf-8'))
    items = response_dict['items']
    for item_index in range(0, len(items)):
      remove_tag = re.compile('<.*?>')
      title = re.sub(remove_tag, '', items[item_index]['title'])
      link = items[item_index]['link']
      image = items[item_index]['image']
      low_price = items[item_index]['lprice']
      high_price= items[item_index]['hprice']
      mall_name = items[item_index]['mallName']
      product_id = items[item_index]['productId']
      product_type = items[item_index]['productType']
      maker = items[item_index]['maker']
      brand = items[item_index]['brand']
      shop_df.loc[idx] = [title, link, image, low_price, high_price, mall_name, product_id, product_type, maker, brand]
      idx += 1
  else:
    print("Error Code:" + rescode)

shop_df

검색 질의: 루이까또즈


HTTPError: ignored

### 위의코드 HTTPError: HTTP Error 401: Unauthorized  이유 알아보기

## 2. 데이터 전처리_weekly 기준으로 쇼핑 트렌드 키워드 정제 

## 3. 데이터 시각화_Tableau로 제작 예정